In [ ]:
!pip install datasets #to access squad dataset
!pip install pyarrow   #to deal with parquet files for saving dataset if required
!pip install tqdm     #for progress bars
!pip install transformers # for t5 model
!pip install tokenizers  #tokenizers from HuggingFace
!pip install sentencepiece #subword tokenizer used by T5
!pip install pytorch-lightning # pytorch wrapper
!pip install torchtext # text utilities
!pip install numpy # text utilities
!pip install evaluate
!pip install rouge
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.2/802.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 17.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none

In [ ]:
#imports
import pandas as pd
import torch
from tqdm import tqdm
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from pprint import pprint
import copy
import json
import numpy as np
import random
import nltk
import evaluate
# import rogue

In [ ]:
device  = 'cuda' if torch.cuda.is_available() else "cpu"

In [ ]:
pd.options.display.max_rows , pd.options.display.max_columns  = 100,100

**Transformer Import**

In [ ]:
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# model_path = "/content/drive/MyDrive/qag_t5_trained_model_v4"
# tknizer_path = "/content/drive/MyDrive/qag_t5_tokenizer_v4"

In [ ]:
# t5_tokenizer = T5Tokenizer.from_pretrained(tknizer_path,model_max_length=512)
# t5_model = T5ForConditionalGeneration.from_pretrained(model_path)
# t5_tokenizer = T5Tokenizer.from_pretrained('google-t5/t5-small',model_max_length=512)
# t5_model = T5ForConditionalGeneration.from_pretrained('google-t5/t5-small')
t5_tokenizer = T5Tokenizer.from_pretrained('google-t5/t5-base')
t5_model = T5ForConditionalGeneration.from_pretrained('google-t5/t5-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

**SQUAD ENG**

In [ ]:
train_dataset = load_dataset('squad', split='train')
valid_dataset = load_dataset('squad', split='validation')
print(f"Total Train Samples:{len(train_dataset)} , Total Validation Samples:{len(valid_dataset)}");

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Total Train Samples:87599 , Total Validation Samples:10570


In [ ]:
sample_validation_dataset = next(iter(valid_dataset))
pprint (sample_validation_dataset)

context = sample_validation_dataset['context']
question = sample_validation_dataset['question']
answer = sample_validation_dataset['answers']['text'][0]
print('---------------'*9)
print('\nBreaking it Down\n')
print ("context:",context)
print ("question:",question)
print ("answer:",answer)

{'answers': {'answer_start': [177, 177, 177],
             'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']},
 'context': 'Super Bowl 50 was an American football game to determine the '
            'champion of the National Football League (NFL) for the 2015 '
            'season. The American Football Conference (AFC) champion Denver '
            'Broncos defeated the National Football Conference (NFC) champion '
            'Carolina Panthers 24–10 to earn their third Super Bowl title. The '
            "game was played on February 7, 2016, at Levi's Stadium in the San "
            'Francisco Bay Area at Santa Clara, California. As this was the '
            '50th Super Bowl, the league emphasized the "golden anniversary" '
            'with various gold-themed initiatives, as well as temporarily '
            'suspending the tradition of naming each Super Bowl game with '
            'Roman numerals (under which the game would have been known as '
            '"Super

In [ ]:
def squad_process(data,answer_threshold=40,verbose = False):
  count_long ,count_short = 0 , 0
  result_df  = pd.DataFrame(columns = ['context', 'answer','question'])
  for index,val in enumerate(tqdm(data)):
      passage = val['context']
      question = val['question']
      answer = val['answers']['text'][0]
      no_of_words = len(answer.split())
      if no_of_words >= answer_threshold:
          count_long = count_long + 1
          continue
      else:
          result_df.loc[count_short] = [passage] + [answer] + [question]
          count_short = count_short + 1
  if verbose:
    return (result_df,
            count_long,
            count_short)
  else:
    return result_df

In [ ]:
df_train , df_validation = squad_process(train_dataset) , squad_process(valid_dataset)
print(f"\n Total Train Samples:{df_train.shape} , Total Validation Samples:{df_validation.shape}")

100%|██████████| 10570/10570 [00:15<00:00, 669.46it/s]


 Total Train Samples:(87598, 3) , Total Validation Samples:(10570, 3)


In [ ]:
df = df_train.copy()
test_df = df[:11877]
df_train = df[11877:]

In [ ]:
test_df

,context,answer,question
0,"Architecturally, the school has a Catholic cha...",Saint Bernadette Soubirous,To whom did the Virgin Mary allegedly appear i...
1,"Architecturally, the school has a Catholic cha...",a copper statue of Christ,What is in front of the Notre Dame Main Building?
2,"Architecturally, the school has a Catholic cha...",the Main Building,The Basilica of the Sacred heart at Notre Dame...
3,"Architecturally, the school has a Catholic cha...",a Marian place of prayer and reflection,What is the Grotto at Notre Dame?
4,"Architecturally, the school has a Catholic cha...",a golden statue of the Virgin Mary,What sits on top of the Main Building at Notre...
...,...,...,...
11872,Notable athletes include swimmer Sharron Davie...,Canadian,What is the current nationality of former Plym...
11873,Notable athletes include swimmer Sharron Davie...,Lyndon B. Johnson,What United States vice president did Donald M...
11874,Notable athletes include swimmer Sharron Davie...,Clear and Present Danger,In what film did Donald Moffat play President ...
11875,Notable athletes include swimmer Sharron Davie...,comedian,What is Dawn French's job?


In [ ]:
SEP_TOKEN = '<sep>'
MASKING_CHANCE = 0.4 #30% chance to replace the answer with '[MASK]'

In [ ]:
class QuestionGenerationDataset(Dataset):
    def __init__(self, tokenizer, filepath, max_len_inp=512,max_len_out=96):
        self.path = filepath

        self.passage_column = "context"
        self.answer = "answer"
        self.question = "question"

        self.data = pd.read_parquet(self.path)
        # self.data = pd.read_parquet(self.path).iloc[:2000,:]

        self.max_len_input = max_len_inp
        self.max_len_output = max_len_out
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []
        self._build()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask = self.inputs[index]["attention_mask"].squeeze()  #squeeze to get rid of the batch dimension
        target_mask = self.targets[index]["attention_mask"].squeeze()  # convert [batch,dim] to [dim]

        labels = copy.deepcopy(target_ids)
        labels [labels==0] = -100

        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask,"labels":labels}

    def _build(self):
        for rownum,val in tqdm(self.data.iterrows()): # Iterating over the dataframe
            passage,target = val[self.passage_column],val[self.question]
            answer = val[self.answer] if np.random.rand() > MASKING_CHANCE else '[MASK]'
            input_ = f"context: {passage}  answer: {answer}" # T5 Input format for question answering tasks
            target = f"question: {str(target)} answer: {val[self.answer]}" # Output format we require
            # target = f"{str(target)}"

            # tokenize inputs
            tokenized_inputs = self.tokenizer.batch_encode_plus(
                [input_], max_length=self.max_len_input,padding='max_length',
                truncation = True,
                return_attention_mask=True,
                add_special_tokens=True,
                return_tensors="pt"
            )
            # tokenize targets
            tokenized_targets = self.tokenizer.batch_encode_plus(
                [target], max_length=self.max_len_output,padding='max_length',
                truncation = True,
                return_attention_mask=True,
                add_special_tokens=True,
                return_tensors="pt"
            )

            self.inputs.append(tokenized_inputs)
            self.targets.append(tokenized_targets)

In [ ]:
# Saving data for future use
df_train.to_parquet('train_squad_v2.parquet')
df_validation.to_parquet('validation_squad_v2.parquet')
test_df.to_parquet('test_squad_v2.parquet')

In [ ]:
# train_path = "/content/drive/MyDrive/train_squad.parquet"
# validation_path = "/content/drive/MyDrive/validation_squad.parquet"

train_path = 'train_squad_v2.parquet' # change this accordingly
validation_path = 'validation_squad_v2.parquet'
test_path = 'test_squad_v2.parquet'
train_dataset = QuestionGenerationDataset(t5_tokenizer,train_path)
validation_dataset = QuestionGenerationDataset(t5_tokenizer,validation_path)
test_dataset = QuestionGenerationDataset(t5_tokenizer,test_path)

75721it [02:22, 532.19it/s]
10570it [00:21, 482.98it/s]
11877it [00:22, 537.13it/s]


In [ ]:
# Data Sample

train_sample = train_dataset[10] # thanks to __getitem__
decoded_train_input = t5_tokenizer.decode(train_sample['source_ids'])
decoded_train_output = t5_tokenizer.decode(train_sample['target_ids'])

print(decoded_train_input)
print(decoded_train_output)

context: According to Titus 3:10 a divisive person should be warned two times before separating from him. The Greek for the phrase "divisive person" became a technical term in the early Church for a type of "heretic" who promoted dissension. In contrast correct teaching is called sound not only because it builds up in the faith, but because it protects against the corrupting influence of false teachers. answer: [MASK]</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [ ]:
import pytorch_lightning as pl
from torch.optim import AdamW
import argparse
from transformers import (
    get_linear_schedule_with_warmup
  )

class T5Tuner(pl.LightningModule):

    def __init__(self,t5model, t5tokenizer,batchsize=16):
        super().__init__()
        self.model = t5model
        self.tokenizer = t5tokenizer
        self.batch_size = batchsize

    def forward(self, input_ids, attention_mask=None,
                decoder_attention_mask=None,
                lm_labels=None):

         outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=decoder_attention_mask,
            labels=lm_labels,
        )

         return outputs

    def training_step(self, batch, batch_idx):
        outputs = self.forward(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_attention_mask=batch['target_mask'],
            lm_labels=batch['labels']
        )

        loss = outputs[0]
        self.log('train_loss',loss)
        return loss

    def validation_step(self, batch, batch_idx):
        outputs = self.forward(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_attention_mask=batch['target_mask'],
            lm_labels=batch['labels']
        )

        loss = outputs[0]
        self.log("val_loss",loss)
        return loss

    def train_dataloader(self):
        return DataLoader(train_dataset, batch_size=self.batch_size,
                          num_workers=2)

    def val_dataloader(self):
        return DataLoader(validation_dataset,
                          batch_size=self.batch_size,
                          num_workers=2)

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=3e-4, eps=1e-8)
        return optimizer

In [ ]:
# from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint

# early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=0.00, patience=3, verbose=True, mode="max")

In [ ]:
checkpoint_callback = ModelCheckpoint(
        dirpath='checkpoints',
        filename='best-checkpoint',
        save_top_k=-1,
        verbose=True,
        monitor='val_loss',
        mode='min'
    )

In [ ]:
model = T5Tuner(t5_model,t5_tokenizer)

trainer = pl.Trainer(callbacks= [checkpoint_callback],max_epochs = 1,accelerator=device)

trainer.fit(model)

In [ ]:
# saving the model
!mkdir "qag_t5_tokenizer_v7"
!mkdir "qag_t5_trained_model_v7"
model.model.save_pretrained('qag_t5_trained_model_v7')
t5_tokenizer.save_pretrained('qag_t5_tokenizer_v7')

In [ ]:
trained_model_path = 'qag_t5_trained_model_v7'
trained_tokenizer = 'qag_t5_tokenizer_v7'

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(trained_model_path)
tokenizer = T5Tokenizer.from_pretrained(trained_tokenizer)

In [ ]:
def get_question(mdl,tknizer,sentence,answer="[MASK]",ref_question=None):

  ''' function to generate questions. Takes a sentence,answer,
      model and tokenizer
  '''

  text = "context: {} answer: {}".format(sentence,answer)
  max_len = 256
  encoding = tknizer.encode_plus(text,max_length=max_len, pad_to_max_length=False,truncation=True, return_tensors="pt")

  input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

  outs = mdl.generate(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  early_stopping=True,
                                  num_beams=10,
                                  num_return_sequences=10,
                                  no_repeat_ngram_size=2,
                                  max_length=72)


  dec = [tknizer.decode(ids,skip_special_tokens=True) for ids in outs]

  Question = dec[0].replace("question:","")
  Question= Question.strip()
  if ref_question:
    bleu = evaluate.load("google_bleu")
    score = bleu.compute(predictions=[Question],
                              references=[ref_question])
    print("***************************************")
    print("\n")
    print("Context: \n", context)
    print("\n")
    print("Predicted Question: \n", dec)
    print("\n")
    print("BLEU: ", score)
    print("***************************************")
  return Question

In [ ]:
import random

In [ ]:
index = random.randrange(1,6000)
context = df_validation["context"].iloc[index]
answer = df_validation["answer"].iloc[index]
question = df_validation["question"].iloc[index]
text = "context: "+context + " " + "\n answer: " + answer + "\n question: " + question
print(text)

In [ ]:
context = "Oxygen is a chemical element; it has symbol O and atomic number 8. It is a member of the chalcogen group in the periodic table, a highly reactive nonmetal, and an oxidizing agent that readily forms oxides with most elements as well as with other compounds. Oxygen is the most abundant element in Earth's crust, and after hydrogen and helium, it is the third-most abundant element in the universe. At standard temperature and pressure, two atoms of the element bind to form dioxygen, a colorless and odorless diatomic gas with the formula . Diatomic oxygen gas currently constitutes 20.95% of the Earth's atmosphere, though this has changed considerably over long periods of time. Oxygen makes up almost half of the Earth's crust in the form of oxides."
answer= "[MASK]"
question = "sdfd"

In [ ]:
ques = get_question(mdl=model,tknizer=tokenizer,sentence=context,answer=answer,ref_question=question)
# print ("question: ", ques)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r "train_squad_v2.parquet" "/content/drive/MyDrive"
!cp -r "validation_squad_v2.parquet" "/content/drive/MyDrive"

In [ ]:
!cp -r "qag_t5_trained_model_v6" "/content/drive/MyDrive"

In [ ]:
!cp -r "qag_t5_tokenizer_v6" "/content/drive/MyDrive"